In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from bs4 import BeautifulSoup
# import re
# import pandas as pd
import os
import time
import re
import shutil

In [2]:
url = 'https://www.symmap.org/browse/'

# Open Firefox and Nav to URL
## NOTE -- Need to modify certain prefs for your system
Will be noted with comments

## Geckodriver
https://github.com/mozilla/geckodriver/releases  
I downloaded and unzipped one of the pre-compiled geckodrivers for my Windows system.  
The executable sits in a random directory, and I point to it when calling the webdriver for Firefox below. You may need to adjust that line.

In [3]:
fp = webdriver.FirefoxProfile()
fp.set_preference('browser.download.folderList', 2) 
fp.set_preference('browser.download.manager.showWhenStarting', False)
# Set the default download dir
fp.set_preference('browser.download.dir', r"C:\\selenium\\")
fp.set_preference('browser.helperApps.neverAsk.openFile', 'text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml')
fp.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml')
fp.set_preference('browser.helperApps.alwaysAsk.force', False)
fp.set_preference('browser.download.manager.alertOnEXEOpen', False)
fp.set_preference('browser.download.manager.focusWhenStarting', False)
fp.set_preference('browser.download.manager.useWindow', False)
fp.set_preference('browser.download.manager.showAlertOnComplete', False)
fp.set_preference('browser.download.manager.closeWhenDone', False)
# Point to the geckodriver you downloaded on your local
driver = webdriver.Firefox(fp, executable_path=r'C:\Users\Ian.Bettinger\AppData\Local\Programs\Python\Python37-32\geckodriver.exe')
driver.window_handles
driver.switch_to_window(driver.window_handles[0])
driver.get(url)

C:\Users\Ian.Bettinger\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: use driver.switch_to.window instead


# Workaround for Herb IDs
## Screw the Pagination and Pull What Exists as of Now
Herb IDs go from 00001 - 00499. We'll just programatically generate the links.

In [4]:
herb_id_dict = dict()

url_head = "https://www.symmap.org/detail/SMHB0"
max_id = 499

for i in range(1, max_id + 1):
    # Store links to disease pages as {disease_id: href} pairs
    id_len = len(str(i))
    padding_size = 4 - id_len
    herb_id_name = 'SMHB0' + padding_size * str(0) + str(i)
    herb_id_dict[herb_id_name] = url_head + padding_size * str(0) + str(i)

In [5]:
herb_id_dict

{'SMHB00001': 'https://www.symmap.org/detail/SMHB00001',
 'SMHB00002': 'https://www.symmap.org/detail/SMHB00002',
 'SMHB00003': 'https://www.symmap.org/detail/SMHB00003',
 'SMHB00004': 'https://www.symmap.org/detail/SMHB00004',
 'SMHB00005': 'https://www.symmap.org/detail/SMHB00005',
 'SMHB00006': 'https://www.symmap.org/detail/SMHB00006',
 'SMHB00007': 'https://www.symmap.org/detail/SMHB00007',
 'SMHB00008': 'https://www.symmap.org/detail/SMHB00008',
 'SMHB00009': 'https://www.symmap.org/detail/SMHB00009',
 'SMHB00010': 'https://www.symmap.org/detail/SMHB00010',
 'SMHB00011': 'https://www.symmap.org/detail/SMHB00011',
 'SMHB00012': 'https://www.symmap.org/detail/SMHB00012',
 'SMHB00013': 'https://www.symmap.org/detail/SMHB00013',
 'SMHB00014': 'https://www.symmap.org/detail/SMHB00014',
 'SMHB00015': 'https://www.symmap.org/detail/SMHB00015',
 'SMHB00016': 'https://www.symmap.org/detail/SMHB00016',
 'SMHB00017': 'https://www.symmap.org/detail/SMHB00017',
 'SMHB00018': 'https://www.symm

# Find the Dropdown and Click to Open

In [7]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

# Click the Disease Link and Download

In [8]:
# Click the dropdown
driver.find_element_by_xpath(r"//button[@data-id='browse_type']").click()

# Wait until the list loads, then click on the fifth list item which is 'diseases'
target = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@class='dropdown-menu inner']/li[@data-original-index='4']")))
target.click()

TimeoutException: Message: 


In [ ]:
# File save & rename paths
# Selenium can't handle file renaming the way this is set up,
# so the workaround is to rename and move the file to another
# directory
filepath = r'c:/selenium'
destination = r'c:/selenium/test/'

# Time Outs to Prevent Banhammer
Each run adds a total of 5 seconds of sleeptime to avoid hammering the server and getting banned.

In [ ]:
# Loop through the dict
for name, link in herb_id_dict.items():
    # Add some wait time to avoid hammering server
    time.sleep(2)
    
    # Pull the link
    driver.get(link)
    
    # Zzzzzzz
    #time.sleep(1)
    
    # Click the dropdown
    driver.find_element_by_xpath(r"//button[@data-id='browse_type']").click()

    # Wait until the list loads, then click on the fifth list item which is 'diseases'
    target = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@class='dropdown-menu inner']/li[@data-original-index='4']")))
    target.click()
    
    # Find and click the download button
    driver.find_element_by_xpath(r"//button[@class='btn btn-primary pull-right']").click()
    
    # Look for the newest file and store the name
    filename = max([filepath +"/"+ f for f in os.listdir(filepath)], key=os.path.getctime)
    
    # Use shutil.move() to rename and move that file to the final dir
    shutil.move(os.path.join(filepath, filename), os.path.join(destination, name + '.csv'))
    
    # Zzzzzzzz
    time.sleep(1)

## Code for Pulling the MM_Symptoms instead of Diseases path done above

In [ ]:
# Click the dropdown
driver.find_element_by_xpath(r"//button[@data-id='browse_type']").click()

# Wait until the list loads, then click on the fifth list item which is 'MMSymtpoms'
target = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@class='dropdown-menu inner']/li[@data-original-index='1']")))
target.click()

In [ ]:
# File save & rename paths
# Selenium can't handle file renaming the way this is set up,
# so the workaround is to rename and move the file to another
# directory
filepath = r'c:/selenium'
destination = r'c:/selenium/symptom_files/'

In [ ]:
# Loop through the dict
for name, link in herb_id_dict.items():
    # Add some wait time to avoid hammering server
    time.sleep(2)
    
    # Pull the link
    driver.get(link)
    
    # Zzzzzzz
    time.sleep(2)
    
    # Click the dropdown
    driver.find_element_by_xpath(r"//button[@data-id='browse_type']").click()

    # Wait until the list loads, then click on the 2nd list item which is 'mm_symptoms'
    target = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//ul[@class='dropdown-menu inner']/li[@data-original-index='1']")))
    target.click()
    
    # Find and click the download button
    driver.find_element_by_xpath(r"//button[@class='btn btn-primary pull-right']").click()
    
    # Look for the newest file and store the name
    filename = max([filepath +"/"+ f for f in os.listdir(filepath)], key=os.path.getctime)
    
    # Use shutil.move() to rename and move that file to the final dir
    shutil.move(os.path.join(filepath, filename), os.path.join(destination, name + '.csv'))
    
    # Zzzzzzzz
    time.sleep(1)